In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from matplotlib import animation
np.random.seed(42)
%load_ext autoreload
%autoreload 2

In [6]:
# loading and unpacking erp data
erp_data = np.load('erp.npz')
erp=erp_data['erp']
timepoints=erp_data['timepoints']
chan_locs_x=erp_data['chan_locs_x']
chan_locs_y=erp_data['chan_locs_y']
conditions = erp_data['conditions']

array(['set size 1', 'set size 3'], dtype='<U10')